!

In [ ]:
%%capture
!pip install --upgrade xarray --quiet
!pip install --upgrade rioxarray --quiet

In [ ]:
import xarray as xr
import rioxarray as rio
from rasterio.warp import reproject, Resampling
import pandas as pd
import numpy as np
import datetime
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

%cd /content/gdrive/MyDrive

Mounted at /content/gdrive/
/content/gdrive/MyDrive


In [ ]:
# functions to calculate dekadal, monthly and seasonal sums from the ET_look output netcdf file

# decadal sum
def dekadal_sum(ds):
    # Store encoding information
    encoding = {var: ds[var].encoding for var in ds.data_vars}

    d = ds.time.dt.day - np.clip((ds.time.dt.day-1) // 10, 0, 2)*10 - 1
    date = ds.time.values - np.array(d, dtype="timedelta64[D]")
    ds['time'] = date
    ds_dk = ds.groupby(ds.time).sum(dim='time', keep_attrs=True)
    # Restore encoding information
    for var in ds_dk.data_vars:
        if var in encoding:  # Ensure the variable exists in the original encoding
            ds_dk[var].encoding = encoding[var]

    return ds_dk

# Monthly sum
def monthly_sum(ds):
    # Store encoding information
    encoding = {var: ds[var].encoding for var in ds.data_vars}

    ds_mn = ds.resample(time="1ME").sum()
    # Restore encoding information
    for var in ds_mn.data_vars:
        if var in encoding:  # Ensure the variable exists in the original encoding
            ds_mn[var].encoding = encoding[var]
    return ds_mn
# Check if the start and end time of the selected dataarray corresponds to sos and eos
def select_season_da(da_var, season_start_date, season_end_date):

    sos = datetime.datetime.fromisoformat(season_start_date) #start of season date, we use datetime.datetime to convert the year, month, day to a datetime object
    eos = datetime.datetime.fromisoformat(season_end_date) #end of season date
    # check if sos and eos are within the time of the datset

    da_st = datetime.datetime.fromisoformat(pd.to_datetime(da_var.time.data).strftime('%Y-%m-%d')[0])
    da_et = datetime.datetime.fromisoformat(pd.to_datetime(da_var.time.data).strftime('%Y-%m-%d')[-1])
    try:
        if (sos >= da_st) or (eos <= da_et):
            da = da_var.sel(time=slice(sos, eos))
            return da
        else:
            print("The sos and/or eos out of the time range of the dataset.")
            da = da_var.sel(time=slice(sos, eos))
            return da
    except ValueError:
        print("Erro in selecting data for the season.")


# Seasonal Resample
def seasonal_sum(ds, sos, eos):
    # Store encoding information
    encoding = {var: ds[var].encoding for var in ds.data_vars}

    ds_sn = select_season_da(ds, sos, eos)
    ds_sn = ds_sn.sum(dim = 'time')
    # Restore encoding information
    for var in ds_sn.data_vars:
        if var in encoding:  # Ensure the variable exists in the original encoding
            encoding[var]['sos']= sos
            encoding[var]['eos']= eos
            ds_sn[var].encoding = encoding[var]
    return ds_sn

# reproject the dataset
def reproject(ds):
  crs_info = input(f"The estimated UTM crs is {ds.rio.estimate_utm_crs()}.\nWould like to repoject the dataset? Enter a valid EPSG code\
  or a template raster file path: ")
  try:
      # Convert it into integer
      to_crs = int(crs_info)
      print(f"project to EPSG:{crs_info}")
      to_crs = f"EPSG:{to_crs}"
      dst = ds.rio.reproject(to_crs)
      # dst = ds.rio.reproject(to_crs, resampling=Resampling.nearest, nodata=-9999)
      return dst
  except ValueError:
      try:
          if os.path.exists(crs_info):
              print("Use a template raster to repoject the dataset")
              temp_rst_file = crs_info
              da_rst = rio.open_rasterio(temp_rst_file)
              if da_rst.rio.crs != None:
                  dst= ds.rio.reproject_match(da_rst, nodata=-9999)
                  return dst
              else:
                  print(f"the template raster {temp_rst_file} does not have CRS information.")
      except ValueError:
          print("Your input is not either a valid EPSG code or a teplate raster path.")


switcher = {
        'et': 'AETI',
        'e': 'E',
        'int': 'I',
        'npp': 'NPP',
        't': 'T',
        'se': 'RSM'
    }
def get_var_name(var_name):
    func = switcher.get(var_name, "nothing")
    # Execute the function
    return func
switcher2 = {
        'dekadal': 'D',
        'monthly': 'M',
        'seasonal': 'S',
    }
def get_time_code(temporal_res):
    func = switcher2.get(temporal_res, "nothing")
    # Execute the function
    return func

# netCDF to geotiff
def write2gtiff(ds, temporal_res, dir_out):

  if 'time' in ds.dims:
      date_str = pd.to_datetime(ds.time.data).strftime('%Y-%m-%d')

  for var in ds.data_vars:
    var_name = get_var_name(var.split('_')[0])
    time_code = get_time_code(temporal_res)
    var_name = f"{var_name}_{time_code}"
    fd = os.path.join(dir_out, temporal_res, f"pywapor_{var_name}")
    encoding  = ds[var].encoding
    attrs = ds[var].attrs
    if(temporal_res.lower() == 'seasonal'):
        sos = encoding['sos']
        eos = encoding['eos']
    # Create folder per variable.
    if not os.path.isdir(fd):
        os.makedirs(fd)

    if(temporal_res.lower() != 'seasonal'):
        for i in range(len(ds.time)):
            date = date_str[i]
            fname = os.path.join(fd, f"pywapor_{var_name}_{date}")
            da = ds[var][i]
            da = da.drop_vars('time')  # get the data for one time step
            # da = da.where(da==encoding['_FillValue'], da*da.attrs['scale_factor'])

            # Modify the attributes
            attrs_to_delete = [j for j in attrs if 'NETCDF_' in j or 'scale_factor' in j]
            attrs_new = {key: attrs[key] for key in attrs if key not in attrs_to_delete}
            lname = attrs_new['long_name']
            if 'Daily' in lname:
              lname = lname.replace("Daily", temporal_res)

            attrs_new.update({'date': date,
                            'units' : f"mm/{temporal_res}",
                            'temporal_resolution' : temporal_res,
                            'long_name': lname
                              })
            da.encoding['scale_factor'] = 1.0
            da.attrs  = attrs_new
            # write the dataarray to a geotif file
            da = da.astype('float32')
            da.rio.to_raster(f"{fname}.tif", driver="GTiff", compress="LZW")
            da.close()
    else:
        date = f"{sos}_{eos}"
        fname = os.path.join(fd, f"pywapor_{var_name}_{date}")
        da = ds[var]
        # da = da.where(da==encoding['_FillValue'], da*da.attrs['scale_factor'])

        # Modify the attributes
        attrs_to_delete = [j for j in attrs if 'NETCDF_' in j or 'scale_factor' in j]
        attrs_new = {key: attrs[key] for key in attrs if key not in attrs_to_delete}
        lname = attrs_new['long_name']
        if 'Daily' in lname:
          lname = lname.replace("Daily", temporal_res)

        attrs_new.update({'date': date,
                        'units' : f"mm/{temporal_res}",
                        'temporal_resolution' : temporal_res,
                        'long_name': lname
                          })
        da.encoding['scale_factor'] = 1.0
        da.attrs  = attrs_new
        # write the dataarray to a geotif file
        da = da.astype('float32')
        # write the dataarray to a geotif file
        da.rio.to_raster(f"{fname}.tif", driver="GTiff", compress="LZW")
        da.close()

#### Step 1: Read pywapor output

In [ ]:
# path to the et_look_out/nc file
path_et_look_out = r'/content/gdrive/MyDrive/pywapor/et_look_out.nc'
xr.set_options(keep_attrs=True)
ds = xr.open_dataset(path_et_look_out, decode_coords="all")
ds = ds.rename({'time_bins': 'time'})
# ds

#### Step 2: Reproject the xarray dataset if needed
The ET_look output is in EPSG:4326, use the next cell to reproject the dataset to other projections, otherwise skip it.

In [ ]:
dst = reproject(ds)

The estimated UTM crs is EPSG:32636.
Would like to repoject the dataset? Enter a valid EPSG code  or a template raster file path: 32636
project to EPSG:32636


#### Step 3: Aggregate to the required timestep (dekadal, monthly or seasonal) and write the result to individual geotiff files per time step

In [ ]:
dir_out = r'/content/gdrive/MyDrive/pywapor' # a folder in your gdrive to save the geotif files
# dir_out = r'pywapor_out' # a folder in colab working directory to save the geotif files

In [ ]:
# aggregate to dekadal timestep
ds_dk = dekadal_sum(dst) # dekadal
temporal_res = 'dekadal'
write2gtiff(ds_dk, temporal_res, dir_out)

In [ ]:
# aggregate to monthly timestep
ds_mn = monthly_sum(dst) # monthly
# dir_out = r'pywapor_out' # folder to save the geotif files
temporal_res = 'monthly'
write2gtiff(ds_mn, temporal_res, dir_out)

In [ ]:
# aggregate to seasonal timestep
season_start_date = '2022-10-05' # start od the season in iso format
season_end_date = '2023-11-28' # end of the season in iso format
ds_sn = seasonal_sum(dst, season_start_date, season_end_date)
# dir_out = r'pywapor_out' # folder to save the geotif files
temporal_res = 'seasonal'
write2gtiff(ds_sn, temporal_res, dir_out)



### Zip and downalod the data folder to your local drive

In [ ]:

!zip -r /content/pywapor.zip /content/gdrive/MyDrive/pywapor_out
from google.colab import files
files.download('/content/pywapor.zip')

  adding: content/gdrive/MyDrive/pywapor_out/ (stored 0%)
  adding: content/gdrive/MyDrive/pywapor_out/dekadal/ (stored 0%)
  adding: content/gdrive/MyDrive/pywapor_out/dekadal/pywapor_I_D/ (stored 0%)
  adding: content/gdrive/MyDrive/pywapor_out/dekadal/pywapor_I_D/pywapor_I_D_2022-10-01.tif (deflated 5%)
  adding: content/gdrive/MyDrive/pywapor_out/dekadal/pywapor_I_D/pywapor_I_D_2022-10-11.tif (deflated 6%)
  adding: content/gdrive/MyDrive/pywapor_out/dekadal/pywapor_I_D/pywapor_I_D_2022-10-21.tif (deflated 96%)
  adding: content/gdrive/MyDrive/pywapor_out/dekadal/pywapor_I_D/pywapor_I_D_2022-11-01.tif (deflated 96%)
  adding: content/gdrive/MyDrive/pywapor_out/dekadal/pywapor_I_D/pywapor_I_D_2022-11-11.tif (deflated 96%)
  adding: content/gdrive/MyDrive/pywapor_out/dekadal/pywapor_I_D/pywapor_I_D_2022-11-21.tif (deflated 96%)
  adding: content/gdrive/MyDrive/pywapor_out/dekadal/pywapor_T_D/ (stored 0%)
  adding: content/gdrive/MyDrive/pywapor_out/dekadal/pywapor_T_D/pywapor_T_D_202

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# # test opening one of the geotiff files
# rst_path = r'd:\Codes\WaPOR\pywapor\pywapor_out\et\et_2022-10-01.tif'
# da = rio.open_rasterio(rst_path)
# da.plot()
# da.close()